In [117]:
import numpy as np
import pandas as pd
from math import sqrt
from scipy import optimize as op
from matplotlib import pyplot as plt

# Data Preprocessing

In [118]:
main_data = pd.read_csv('./04_cricket_1999to2011.csv')

data = main_data[main_data['Innings']==1]
data = data[['Over','Runs','Total.Out']]
# Nx50 Matrix holding history data 
Innings1 = list()

inning = np.zeros(100)
i=0
for d in data.itertuples():
    if d[1]==1 or i==50:
        while i!=50:
            inning[i],inning[i+50] = inning[i-1],inning[50+i-1]
            i+=1
        if inning[49]<400 and inning[49]>150 and inning[50]>=0  and inning[-1]<=10 and np.all(np.diff(inning[:50])>=0) and np.all(np.diff(inning[50:])>=0):
            Innings1.append(inning)
        inning = np.zeros(100)
        i=0
    inning[i],inning[i+50] = inning[i-1]+d[2],d[3]
    i+=1

Innings1 = np.array(Innings1)     
Innings1 = Innings1[1:,:]
print(Innings1.shape)


(1274, 100)


## mRSC Algorithm

In [151]:
def mRSC(X,data,intervention_point,k,W):
    U,S,V = np.linalg.svd(data)
    S_ = np.diag(S[:k])
    M = U[:,:k]@S_@V[:k,:]    

    M_k = np.zeros(shape=(M.shape[0],2*intervention_point))
    M_k[:,:intervention_point],M_k[:,intervention_point:] = M[:,:intervention_point],W*M[:,50:50+intervention_point]
    X[intervention_point:] = W*X[intervention_point:] 
    
    opt_result = op.least_squares(lambda v : X-v@M_k,np.zeros(M_k.shape[0]))
    xopt = opt_result.x
    result = np.int16(np.round(xopt@M))
    return result,xopt

## Task 1 : Estimate Deviations in Predictions of 1st Innings

In [123]:
np.random.shuffle(Innings1)
P = int(Innings1.shape[0]*0.8)
train,test = Innings1[:P,:],Innings1[P:,:]

In [ ]:
error = np.zeros(50)
for inning in test[np.random.randint(0,test.shape[0],50),:]:
    for intervention_point in range(50):
        Z = np.zeros(2*intervention_point)
        Z[:intervention_point],Z[intervention_point:] = inning[:intervention_point],inning[50:50+intervention_point]
        prediction = mRSC(Z,train,intervention_point,4)
        error[intervention_point] += abs(inning[49]-prediction[49])
error /= 50
plt.plot(error)

# Task 2 : Prediction

In [154]:
def is_valid(inning):
    if inning[49]<450 and inning[49]>150 and inning[-1]<=10 and inning[50] >=0 and np.all(np.diff(inning[:50])>=0) and np.all(np.diff(inning[50:])>=0):
        return True 
    else:
        return False

error = np.zeros(30)
i=0
matches = test[np.random.randint(0,test.shape[0],5),:]
for inning in matches:
    intervention_point = 30
    Z = np.zeros(2*intervention_point)
    Z[:intervention_point],Z[intervention_point:] = inning[:intervention_point],inning[50:50+intervention_point]
    Wopt = 1
    loss = np.inf
    match = np.zeros(100)
    for W in np.linspace(0.5,5,25):
        prediction,xopt = mRSC(Z,train,intervention_point,10,W)
        A = np.zeros(2*intervention_point)
        A[:intervention_point],A[intervention_point:] = prediction[:intervention_point],prediction[50:50+intervention_point]
        diff1 = sqrt(np.mean(np.square(A[:intervention_point]-Z[:intervention_point])))
        diff2 = sqrt(np.mean(np.square(A[intervention_point:]-Z[intervention_point:])))
        if diff1+W*diff2 <= loss and is_valid(prediction):
            Wopt = W
            loss = diff1+W*diff2
            match = prediction
    print(loss)
    print(np.min(xopt))
    print(match)
    print(inning)
    print(Wopt)
    #error[i] += abs(prediction[49]-inning[49])
    #error[i]/=30
    #print(f'W:{W:.3f} error:{error[i]:.3f}')
    #i+=1
#print(error)
#er,bin_centers = np.histogram(error,bins=50)
# plt.cla()
# plt.scatter(bin_centers[1:],er)
# plt.show()

inf
-40813905.4556337
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[  2.  10.  16.  27.  37.  45.  45.  51.  56.  64.  73.  77.  83.  85.
  96.  98. 103. 104. 106. 107. 108. 110. 116. 119. 122. 124. 129. 132.
 135. 139. 144. 152. 152. 153. 160. 164. 164. 168. 169. 169. 170. 174.
 179. 182. 188. 191. 194. 195. 201. 204.   0.   0.   0.   0.   0.   0.
   1.   1.   1.   2.   2.   2.   2.   2.   3.   3.   3.   3.   4.   4.
   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   5.   6.   6.
   6.   6.   7.   7.   8.   8.   8.   8.   8.   8.   8.   8.   9.   9.
   9.   9.]
1
inf
-58634810.62475694
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0